In [7]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [8]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
HYGVol = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Adj HYG")
LQDyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="LQD")
HYGyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HYG")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")

portfolio = portfolio.set_index("Date")
HYGVol = HYGVol.set_index("Date").sort_index()
HYGVol = HYGVol.loc[:,"Volume"]
LQDyas = LQDyas.set_index("Date").sort_index()
HYGyas = HYGyas.set_index("Date").sort_index()
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()

portfolio.loc[:,"HYGCumDiv"]=portfolio.loc[:,"HYG Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueHYG"]=portfolio.loc[:,"HYG Position"]+portfolio.loc[:,"HYGCumDiv"]
portfolio = portfolio.sort_index()

HYGCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueHYG"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
HYGCorrData.loc[:,"TotalReturnsHYG"] = HYGCorrData["TotalValueHYG"].pct_change()
HYGCorrData = HYGCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS"})
HYGCorrData.loc[:,"HY_Index_OAS_Diff"]=HYGCorrData["HY_Index_OAS"].diff()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
HYGCorrData.loc[:,"10yYieldDiff"] = HYGCorrData["10y_UST_INDEX_PX"].diff()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_Spread"})
HYGCorrData.loc[:,"IG_Index_Z_Spread_Diff"]=(HYGCorrData["IG_INDEX_Z_Spread"].diff())

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGVol.loc["2021-01-11":"2025-12-31"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"Volume":"HYG_Volume"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(portfolio.loc["2021-01-11":"2025-12-31","SPY Position"]/-10,left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"SPY Position":"SPY_LAST_PX"})
HYGCorrData.loc[:,"ReturnSPY"] = HYGCorrData["SPY_LAST_PX"].pct_change()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGyas.loc["2021-01-11":"2025-12-31","YAS_MOD_DUR"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"YAS_MOD_DUR":"HYG_MOD_DUR_YAS"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"Vix Idx"})

HYGCorrData=HYGCorrData.dropna()
HYGCorrData = HYGCorrData.sort_index(ascending=False)


In [11]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3","PC4"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.37731438 0.20546493 0.1981256  0.17009933 0.04899577]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPY  \
0           0.665575     -0.351903                0.652769   0.002016   
1          -0.015731      0.171472                0.009455   0.633909   
2          -0.038276     -0.364072               -0.089255   0.748259   
3           0.209303      0.842593                0.285963   0.195545   
4           0.715185      0.065190               -0.695746  -0.005114   

   MOVE Idx  
0  0.084042  
1  0.753936  
2 -0.546012  
3 -0.355269  
4  0.013121  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     370.6
Date:                Tue, 13 Jan 2026   Prob (F-statistic):          1.08e-209
Time:                        22:43:38   Log-Likelihood:                 5469.5
No. Observations:                1240   AIC:                        -1.093e+04
Df Residuals:                    1235   BIC:                        -1.090e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   8.36e-05      1.570      0.117   -3.28e-05       0.000
PC1           -0.0011   6.09e-05    -17.674      0.000      -0.001      -0.001
PC2           -0.0003   8.25e-05     -3.853      0.000      -0.000      -0.000
PC3            0.0010    8.4e-05     12.128      0.000       0.001       0.001
PC4           -0.0029   9.07e-05    -31.752      0.000      -0.003      -0.003
==============================================================================
Omnibus:                      110.582   Durbin-Watson:                   2.309
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              638.352
Skew:                          -0.134   Prob(JB):                    2.42e-139
Kurtosis:                       6.505   Cond. No.                         1.49
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["ReturnSPY","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3","PC4"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.38911845 0.21019784 0.20070101 0.15101575 0.04896695]
   ReturnSPY   Vix Idx  10yYieldDiff  IG_Index_Z_Spread_Diff  \
0   0.004833  0.256656     -0.312106                0.643158   
1   0.071995  0.750351      0.654603                0.048524   
2   0.989779  0.019828     -0.129357               -0.048497   
3   0.122888 -0.608530      0.673369                0.306587   
4  -0.005332  0.019978      0.062651               -0.698316   

   HY_Index_OAS_Diff  
0           0.650417  
1          -0.030493  
2          -0.029296  
3           0.259168  
4           0.712742  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     370.4
Date:                Tue, 13 Jan 2026   Prob (F-statistic):          1.32e-209
Time:                        22:43:52   Log-Likelihood:                 5469.3
No. Observations:                1240   AIC:                        -1.093e+04
Df Residuals:                    1235   BIC:                        -1.090e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   8.36e-05      1.570      0.117   -3.28e-05       0.000
PC1           -0.0011      6e-05    -18.872      0.000      -0.001      -0.001
PC2           -0.0016   8.16e-05    -19.394      0.000      -0.002      -0.001
PC3            0.0004   8.35e-05      5.254      0.000       0.000       0.001
PC4           -0.0026   9.62e-05    -26.865      0.000      -0.003      -0.002
==============================================================================
Omnibus:                      110.564   Durbin-Watson:                   2.316
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              635.574
Skew:                          -0.138   Prob(JB):                    9.70e-139
Kurtosis:                       6.496   Cond. No.                         1.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["ReturnSPY","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.47007226 0.25088568 0.21776521 0.06127684]
   ReturnSPY  10yYieldDiff  IG_Index_Z_Spread_Diff  HY_Index_OAS_Diff
0  -0.000493     -0.356974                0.653514           0.667450
1   0.986589     -0.151905               -0.051723          -0.029873
2   0.163160      0.919316                0.295098           0.202864
3  -0.004608      0.065973               -0.695098           0.715866


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     492.5
Date:                Tue, 13 Jan 2026   Prob (F-statistic):          1.88e-210
Time:                        22:45:02   Log-Likelihood:                 5468.0
No. Observations:                1240   AIC:                        -1.093e+04
Df Residuals:                    1236   BIC:                        -1.091e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   8.37e-05      1.569      0.117   -3.29e-05       0.000
PC1           -0.0011    6.1e-05    -17.585      0.000      -0.001      -0.001
PC2            0.0005   8.35e-05      5.965      0.000       0.000       0.001
PC3           -0.0030   8.97e-05    -33.653      0.000      -0.003      -0.003
==============================================================================
Omnibus:                      108.026   Durbin-Watson:                   2.305
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              645.265
Skew:                          -0.045   Prob(JB):                    7.63e-141
Kurtosis:                       6.533   Cond. No.                         1.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""